# Less Naive representation pipeline (pennylane)

Main steps:

* input: image of the sky (frequency domain)
* apply 2D Fourier transform --> visibilities (Fourier domain)
* encode classical visibilities data into qbits (non-conventional domain)
* measure qbits (back to conventional domain)
* apply 2D Inverse Fourier transform --> original image?

In [21]:
import numpy as np
import math
import sys

import pennylane as qml
from pennylane.optimize import AdamOptimizer, GradientDescentOptimizer
from pennylane.templates.embeddings import AmplitudeEmbedding

## Generating an artificial image of the sky

In [22]:
#image of the sky filled with single precision complex floats
#pixels are set to low complex random values (image background) 
#few pixels are set to larger complex random values in a specified ellipse area (image source/subject)

n = 4
image = np.zeros((n, n), dtype=complex)
image.real = np.random.rand(n , n) / 100
image.imag = np.random.rand(n , n) / 100

h, w = image.shape
mask = circular_mask(h, w, radius=h/2)
sky_image = image.copy()
sky_image[~mask] = complex(np.random.rand() * 100, np.random.rand() * 100)
print(sky_image)

[[9.03018518e+01+7.13215916e+01j 9.03018518e+01+7.13215916e+01j
  6.12378151e-03+5.82795563e-04j 9.03018518e+01+7.13215916e+01j]
 [9.03018518e+01+7.13215916e+01j 2.56325549e-03+6.65787144e-03j
  6.29310156e-03+9.05023573e-03j 3.06964087e-03+9.50770866e-03j]
 [9.13070683e-04+6.11993091e-03j 7.38098334e-03+7.06396292e-03j
  8.97665017e-03+7.08359559e-03j 1.96924033e-03+4.70749384e-03j]
 [9.03018518e+01+7.13215916e+01j 8.18975824e-03+7.20385564e-03j
  8.85601289e-03+3.38293620e-04j 8.66337052e-04+4.61125914e-03j]]


## Applying 2D Fourier transform (visibilities)

In [23]:
visibilities = np.fft.fft2(sky_image)
print(visibilities)

[[ 451.56446083+356.67088496j  270.87831815+213.94161101j
    90.3089752  +71.30501429j  270.87411969+213.96606857j]
 [ 270.90550164+213.9463687j    90.30055299 +71.33026373j
   -90.29190604 -71.32330115j   90.28960632 +71.30855537j]
 [  90.29737742 +71.30978015j  -90.29827634 -71.31234423j
  -270.89935224-213.94416775j  -90.30350376 -71.31515482j]
 [ 270.87937683+213.93439648j   90.30231726 +71.32450468j
   -90.30062899 -71.32158058j   90.32268982 +71.32456606j]]


### Sanity check

In [24]:
img = np.fft.ifft2(visibilities)
print(img)
test_real = ((sky_image.real - img.real)**2).mean()
test_imag = ((sky_image.imag - img.imag)**2).mean()

print()
 
print('Reals MSE: '+ str(test_real))
print('Imaginaries MSE: ' + str(test_imag))

[[9.03018518e+01+7.13215916e+01j 9.03018518e+01+7.13215916e+01j
  6.12378151e-03+5.82795563e-04j 9.03018518e+01+7.13215916e+01j]
 [9.03018518e+01+7.13215916e+01j 2.56325549e-03+6.65787144e-03j
  6.29310156e-03+9.05023573e-03j 3.06964087e-03+9.50770866e-03j]
 [9.13070683e-04+6.11993091e-03j 7.38098334e-03+7.06396292e-03j
  8.97665017e-03+7.08359559e-03j 1.96924033e-03+4.70749384e-03j]
 [9.03018518e+01+7.13215916e+01j 8.18975824e-03+7.20385564e-03j
  8.85601289e-03+3.38293620e-04j 8.66337052e-04+4.61125914e-03j]]

Reals MSE: 7.377717416395818e-29
Imaginaries MSE: 1.7509392055706546e-29


## Classical data encoding/decoding

In [32]:
# Amplitude embedding encodes a normalized 2^n-dimensional feature vector into the state of n qbits (uses log2(n) qbits for n classical data)
data_dimension = visibilities.shape[0]*visibilities.shape[1]
data = visibilities.flatten()

#normalization to prepare a qstate with measurement probabilites summing up to 1 (SUM (amplitudes²) = 1)
norm = qml.math.sum(qml.math.abs(data) ** 2)
normalized_data = data / qml.math.sqrt(norm)

wires = range(int(math.log2(data_dimension))) # set the number of qbits (no padding needed if outputs an integer)
amp_dev = qml.device('default.qubit', wires, shots=1)

@qml.qnode(amp_dev)
def amp_encoding(data):
    AmplitudeEmbedding(data, wires) # normalize = True
    return qml.sample(qml.PauliX(0)), qml.sample(qml.PauliX(1)), qml.sample(qml.PauliX(2)), qml.sample(qml.PauliX(3))

result = amp_encoding(normalized_data)
print(result)
readout = amp_dev.state

readout = readout*qml.math.sqrt(norm) # denormalization of the measurement outcomes
readout = np.array(readout).reshape(n , n)
print(readout)

[[-1]
 [-1]
 [ 1]
 [ 1]]
[[ 451.56446083+356.67088496j  270.87831815+213.94161101j
    90.3089752  +71.30501429j  270.87411969+213.96606857j]
 [ 270.90550164+213.9463687j    90.30055299 +71.33026373j
   -90.29190604 -71.32330115j   90.28960632 +71.30855537j]
 [  90.29737742 +71.30978015j  -90.29827634 -71.31234423j
  -270.89935224-213.94416775j  -90.30350376 -71.31515482j]
 [ 270.87937683+213.93439648j   90.30231726 +71.32450468j
   -90.30062899 -71.32158058j   90.32268982 +71.32456606j]]


## Applying 2D Inverse Fourier transform (+ fidelity test)

In [33]:
img = np.fft.ifft2(readout)

test_real = ((sky_image.real - img.real)**2).mean()
test_imag = ((sky_image.imag - img.imag)**2).mean()

print()

print('Reals MSE: '+ str(test_real))
print('Imaginaries MSE: ' + str(test_imag))


Reals MSE: 7.722935544102369e-29
Imaginaries MSE: 2.630356133070539e-29


## Utils

### Quantum

### Classical

In [14]:
#creates a circular mask over a 2D array
def circular_mask(h, w, center=None, radius=None):
    if center is None: #image center
        center = (int(w/2), int(h/2))
    if radius is None: #smallest distance between center and image bounderies
        radius = min(center[0], center[1], w-center[0], h-center[1])
        
    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)
    mask = dist_from_center <= radius
    
    return mask